In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker

from torch.nn.parameter import Parameter
import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat
#Set default dtype to float32
torch.set_default_dtype(torch.float)

from smt.sampling_methods import LHS
#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:2


In [2]:
data = scipy.io.loadmat('burgers_shock_10sin.mat')
label = "QCRE_2D_5_tanhxtanh_NW"
                     
x_test = data['x']                                   # 256 points between -1 and 1 [256x1]
t_test = data['t']   
usol = data['usol']
X_test, T_test = np.meshgrid(x_test,t_test)  

xt_test_tensor = torch.from_numpy(np.hstack((X_test.flatten()[:,None], T_test.flatten()[:,None]))).float().to(device)

u_true = usol.flatten('F')[:,None]

In [3]:
# data = scipy.io.loadmat('burgers_shock_10sin.mat')  	# Load data from file

pi = torch.from_numpy(np.array(np.pi)).double().to(device)

label = "QCRE_2D_5_tanhxtanh_NW"
# x = data['x']                                   # 256 points between -1 and 1 [256x1]
# t = data['t']                                   # 100 time points between 0 and 0.2 [100x1] 
# usol = data['usol']   

#usol = usol/1000# solution of 256x100 grid points

x = np.linspace(-1,1,500).reshape(-1,1)
t = np.linspace(0,0.2,500).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

#y_true = true_2D_1(xt)

bound_pts_1 = (X==-1).reshape(-1,)
xt_bound_1 = xt[bound_pts_1,:]
u_bound_1 = np.zeros((np.shape(xt_bound_1)[0],1))

bound_pts_2 = (X==1).reshape(-1,)
xt_bound_2 = xt[bound_pts_2,:]
u_bound_2 = np.zeros((np.shape(xt_bound_2)[0],1))

bound_pts_3 = (T==0).reshape(-1,)
xt_bound_3 = xt[bound_pts_3,:]
u_bound_3 = -10*np.sin(np.pi*xt_bound_3[:,0].reshape(-1,1))
#u_bound_3 = -10*np.ones((np.shape(bound_pts_3)[0],1))


xt_bound = np.vstack((xt_bound_1,xt_bound_2,xt_bound_3))
u_bound = np.vstack((u_bound_1,u_bound_2,u_bound_3))


lb_xt = xt[0]
ub_xt = xt[-1]

In [ ]:
plt.plot(xt_bound_3[:,0],u_bound_3)

In [4]:
def trainingdata(N_u,N_f,seed):

    '''Boundary Conditions'''
    #Initial Condition -1 =< x =<1 and t = 0  
#     leftedge_x = np.hstack((X[0,:][:,None], T[0,:][:,None])) #L1
#     leftedge_u = usol[:,0][:,None]

#     #Boundary Condition x = -1 and 0 =< t =<1
#     bottomedge_x = np.hstack((X[:,0][:,None], T[:,0][:,None])) #L2
#     bottomedge_u = usol[-1,:][:,None]

#     #Boundary Condition x = 1 and 0 =< t =<1
#     topedge_x = np.hstack((X[:,-1][:,None], T[:,0][:,None])) #L3
#     topedge_u = usol[0,:][:,None]

#     all_X_u_train = np.vstack([leftedge_x, bottomedge_x, topedge_x]) # X_u_train [456,2] (456 = 256(L1)+100(L2)+100(L3))
#     all_u_train = np.vstack([leftedge_u, bottomedge_u, topedge_u])   #corresponding u [456x1]

    #choose random N_u points for training
    np.random.seed(seed)
    idx = np.random.choice(xt_bound.shape[0], N_u, replace=True) 

    X_u_train = xt_bound[idx, :] #choose indices from  set 'idx' (x,t)
    u_train = u_bound[idx,:]      #choose corresponding u

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    X_f_train = lb_xt + (ub_xt-lb_xt)*samples 
    
    X_f_train = np.vstack((X_f_train, X_u_train)) # append training points to collocation points 

    return X_f_train, X_u_train, u_train

In [6]:
#X_u_test = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
X_u_test = xt_test_tensor
# Domain bounds
lb = X_u_test[0]  # [-1. 0.]
ub = X_u_test[-1] # [1.  0.99]

'''
   Fortran Style ('F') flatten,stacked column wise!
   u = [c1 
        c2
        .
        .
        cn]

   u =  [25600x1] 
'''
#u_true = usol.flatten('F')[:,None]

"\n   Fortran Style ('F') flatten,stacked column wise!\n   u = [c1 \n        c2\n        .\n        .\n        cn]\n\n   u =  [25600x1] \n"

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        '''
        Alternatively:
        
        *all layers are callable 
    
        Simple linear Layers
        self.fc1 = nn.Linear(2,50)
        self.fc2 = nn.Linear(50,50)
        self.fc3 = nn.Linear(50,50)
        self.fc4 = nn.Linear(50,1)
        
        '''
    
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            # weights from a normal distribution with 
            # Recommended gain value for tanh = 5/3?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        self.beta = Parameter(1*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.train_loss = []
        self.test_loss = []
        self.beta_val = []
        
        
    'foward pass'
    def forward(self,x):
         if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
         u_b = torch.from_numpy(ub_xt).float().to(device)
         l_b = torch.from_numpy(lb_xt).float().to(device)
        
            
         #preprocessing input 
         x = (x - l_b)/(u_b - l_b) #feature scaling
        
         #convert to float
         a = x.float()
                        
         '''     
         Alternatively:
        
         a = self.activation(self.fc1(a))
         a = self.activation(self.fc2(a))
         a = self.activation(self.fc3(a))
         a = self.fc4(a)
         
         '''
        
         for i in range(len(layers)-3):
                z = self.linears[i](a)
                a = self.activation(z) + self.beta[:,i]*z*self.activation(z)
         
         a = self.linears[-2](a)
         a = self.linears[-1](a)   
            
        
         return a
                        
    def loss_BC(self,x,y):
                
        loss_u = self.loss_function(self.forward(x), y)
                
        return loss_u
    
    def loss_PDE(self, x_to_train_f,f_hat):
        
        nu = 0.01/pi
                
        x_1_f = x_to_train_f[:,[0]]
        x_2_f = x_to_train_f[:,[1]]
                        
        g = x_to_train_f.clone()
                        
        g.requires_grad = True
        
        u = self.forward(g)
                
        u_x_t = autograd.grad(u,g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
                                
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(x_to_train_f.shape).to(device), create_graph=True)[0]
                                                            
        u_x = u_x_t[:,[0]]
        
        u_t = u_x_t[:,[1]]
        
        u_xx = u_xx_tt[:,[0]]
                                        
        f = u_t + (self.forward(g))*(u_x) - (nu)*u_xx 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,x,y,x_to_train_f,f_hat):

        loss_u = self.loss_BC(x,y)
        loss_f = self.loss_PDE(x_to_train_f,f_hat)

        
        
        loss_val = loss_u + loss_f
        
        return loss_val
     
    'callable for optimizer'                                       
    def closure(self):
        optimizer.zero_grad()
        
#         X_f_train_np_array, X_u_train_np_array, u_train_np_array = trainingdata(N_u,N_f,self.iter*32)
        
#         X_f_train = torch.from_numpy(X_f_train_np_array).float().to(device)
#         X_u_train = torch.from_numpy(X_u_train_np_array).float().to(device)
#         u_train = torch.from_numpy(u_train_np_array).float().to(device)
        
#     #u = torch.from_numpy(u_true).float().to(device)
#         f_hat = torch.zeros(X_f_train.shape[0],1).to(device)
        

        loss = self.loss(X_u_train, u_train, X_f_train,f_hat)
        
        self.train_loss.append(loss.cpu().detach().numpy())
        
        
        u_pred = self.test(xt_test_tensor)
        self.test_loss.append(np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1))))
        self.beta_val.append(self.beta.cpu().detach().numpy())
        
        print(self.iter,"Train Loss",self.train_loss[-1],"Test Loss",self.test_loss[-1])
#         print(self.iter,"Train Loss",self.train_loss[-1])
        loss.backward()
        self.iter += 1
     
        return loss        
    'test neural network'
    
    def test(self,xt_test_tensor):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

In [ ]:
xt_test_tensor

In [7]:
max_reps = 1

train_loss_full = []
test_loss_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))


for reps in range(max_reps):
    print(reps)
    N_u = 1000 #Total number of data points for 'u'
    N_f = 10000 #Total number of collocation points 
    
    X_f_train_np_array, X_u_train_np_array, u_train_np_array = trainingdata(N_u,N_f,reps*32)
        
    X_f_train = torch.from_numpy(X_f_train_np_array).float().to(device)
    X_u_train = torch.from_numpy(X_u_train_np_array).float().to(device)
    u_train = torch.from_numpy(u_train_np_array).float().to(device)
        
    #u = torch.from_numpy(u_true).float().to(device)
    f_hat = torch.zeros(X_f_train.shape[0],1).to(device)
        

    #X_u_test_tensor = torch.from_numpy(X_u_test).float().to(device)
    'Convert to tensor and send to GPU'


    layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    #layers = np.array([2,512,512,1])
    PINN = Sequentialmodel(layers)
       
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    'L-BFGS Optimizer'
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                                  max_iter = 10000, 
                                  max_eval = None, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')

    start_time = time.time()

    optimizer.step(PINN.closure)
    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(PINN.train_loss)
    test_loss_full.append(PINN.test_loss)
    elapsed_time[reps] = time.time() - start_time
    beta_full.append(PINN.beta_val)
    

mdic = {"train_loss": train_loss_full,"test_loss": test_loss_full,"Time": elapsed_time, "beta": beta_full, "label": label}
savemat(label+'.mat', mdic) 

0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0 Train Loss 20.604023 Test Loss 24.643764922173155
1 Train Loss 19.652573 Test Loss 24.605853098306127
2 Train Loss 17.86755 Test Loss 24.4661022311603
3 Train Loss 17.702892 Test Loss 24.413271316071917
4 Train Loss 17.521772 Test Loss 24.335553003139793
5 Train Loss 17.284925 Test Loss 24.1841171

144 Train Loss 9.989927 Test Loss 12.990797617374344
145 Train Loss 9.986322 Test Loss 12.980303352110102
146 Train Loss 9.982006 Test Loss 12.989038170602647
147 Train Loss 9.976697 Test Loss 12.991739211612499
148 Train Loss 9.971518 Test Loss 12.982386080180365
149 Train Loss 9.96719 Test Loss 12.980250106026215
150 Train Loss 9.962116 Test Loss 12.978915506769244
151 Train Loss 9.95335 Test Loss 12.965455972570618
152 Train Loss 9.950003 Test Loss 12.954867894983668
153 Train Loss 9.946561 Test Loss 12.956335204847443
154 Train Loss 9.944464 Test Loss 12.947631103162404
155 Train Loss 9.941664 Test Loss 12.9451395789538
156 Train Loss 9.931007 Test Loss 12.929704698472475
157 Train Loss 9.918931 Test Loss 12.911819337228833
158 Train Loss 9.903914 Test Loss 12.901654516170694
159 Train Loss 9.8906555 Test Loss 12.878814607308412
160 Train Loss 9.881534 Test Loss 12.883576276118422
161 Train Loss 9.874165 Test Loss 12.864828300117665
162 Train Loss 9.868316 Test Loss 12.859148219177

300 Train Loss 9.202213 Test Loss 12.166253485185083
301 Train Loss 9.200333 Test Loss 12.163759116503137
302 Train Loss 9.197249 Test Loss 12.160045675130807
303 Train Loss 9.193637 Test Loss 12.157877140627118
304 Train Loss 9.1905365 Test Loss 12.1608198157469
305 Train Loss 9.18756 Test Loss 12.160086409112004
306 Train Loss 9.184197 Test Loss 12.163806283261337
307 Train Loss 9.18053 Test Loss 12.157755820617856
308 Train Loss 9.177009 Test Loss 12.15838854059773
309 Train Loss 9.17333 Test Loss 12.152364351957733
310 Train Loss 9.169933 Test Loss 12.144333480329935
311 Train Loss 9.167769 Test Loss 12.15034218985087
312 Train Loss 9.164641 Test Loss 12.149856634771018
313 Train Loss 9.159487 Test Loss 12.149001516061658
314 Train Loss 9.155525 Test Loss 12.158392719713811
315 Train Loss 9.150526 Test Loss 12.148687313388313
316 Train Loss 9.147721 Test Loss 12.137293575835919
317 Train Loss 9.146586 Test Loss 12.136261010669264
318 Train Loss 9.1452675 Test Loss 12.13375903487996

456 Train Loss 8.812113 Test Loss 11.845291726207249
457 Train Loss 8.810302 Test Loss 11.845274072389484
458 Train Loss 8.807956 Test Loss 11.843051661819196
459 Train Loss 8.8056345 Test Loss 11.840133981731391
460 Train Loss 8.803664 Test Loss 11.840970931960804
461 Train Loss 8.802772 Test Loss 11.838770266137121
462 Train Loss 8.80193 Test Loss 11.836558073093954
463 Train Loss 8.800756 Test Loss 11.833786331409092
464 Train Loss 8.798947 Test Loss 11.831467265351266
465 Train Loss 8.796739 Test Loss 11.828936053360826
466 Train Loss 8.794729 Test Loss 11.827911321211594
467 Train Loss 8.793248 Test Loss 11.829751437372423
468 Train Loss 8.791854 Test Loss 11.829027351481288
469 Train Loss 8.790713 Test Loss 11.831169816725199
470 Train Loss 8.789423 Test Loss 11.828887384984753
471 Train Loss 8.788265 Test Loss 11.826281511664506
472 Train Loss 8.787364 Test Loss 11.823930671380603
473 Train Loss 8.786347 Test Loss 11.82113306343018
474 Train Loss 8.7846985 Test Loss 11.817593711

612 Train Loss 8.55401 Test Loss 11.539216604492596
613 Train Loss 8.553268 Test Loss 11.540559313716322
614 Train Loss 8.552425 Test Loss 11.542318969831479
615 Train Loss 8.551632 Test Loss 11.543507852619923
616 Train Loss 8.551079 Test Loss 11.544546472384729
617 Train Loss 8.550542 Test Loss 11.544185083894646
618 Train Loss 8.549882 Test Loss 11.542632700300457
619 Train Loss 8.549029 Test Loss 11.541391497623618
620 Train Loss 8.547797 Test Loss 11.540646648352785
621 Train Loss 8.546483 Test Loss 11.540826105332348
622 Train Loss 8.545293 Test Loss 11.542206982018307
623 Train Loss 8.543288 Test Loss 11.543961294373318
624 Train Loss 8.542027 Test Loss 11.545175076534509
625 Train Loss 8.541339 Test Loss 11.544338560957199
626 Train Loss 8.540673 Test Loss 11.544169429785047
627 Train Loss 8.540071 Test Loss 11.542654462743617
628 Train Loss 8.539241 Test Loss 11.541440171890988
629 Train Loss 8.537896 Test Loss 11.539462214144894
630 Train Loss 8.536117 Test Loss 11.5371190163

...

2163 Train Loss 8.181015 Test Loss 11.33246810913484
2164 Train Loss 8.180558 Test Loss 11.331722635306841
2165 Train Loss 8.1801605 Test Loss 11.33080614336486
2166 Train Loss 8.180013 Test Loss 11.33031392454963
2167 Train Loss 8.179899 Test Loss 11.330081338023588
2168 Train Loss 8.179735 Test Loss 11.330068565720904
2169 Train Loss 8.201286 Test Loss 11.322619076533533
2170 Train Loss 8.179695 Test Loss 11.329720906641764
2171 Train Loss 8.180068 Test Loss 11.331281279886944
2172 Train Loss 8.179269 Test Loss 11.330394722159177
2173 Train Loss 8.179022 Test Loss 11.33140492248375
2174 Train Loss 8.178754 Test Loss 11.331990179305667
2175 Train Loss 8.178313 Test Loss 11.332117241564701
2176 Train Loss 8.178538 Test Loss 11.332562631499783
2177 Train Loss 8.178182 Test Loss 11.332285391193828
2178 Train Loss 8.177703 Test Loss 11.33266925879176
2179 Train Loss 8.177145 Test Loss 11.333376546815352
2180 Train Loss 8.178025 Test Loss 11.333631281596238
2181 Train Loss 8.176875 Test Lo

2316 Train Loss 8.1294155 Test Loss 11.32224745990761
2317 Train Loss 8.128221 Test Loss 11.319451131200145
2318 Train Loss 8.14523 Test Loss 11.316050845604575
2319 Train Loss 8.127937 Test Loss 11.31906334988455
2320 Train Loss 8.127621 Test Loss 11.319051878549239
2321 Train Loss 8.127333 Test Loss 11.31912016117796
2322 Train Loss 8.127023 Test Loss 11.320496020560478
2323 Train Loss 8.126353 Test Loss 11.320865831449485
2324 Train Loss 8.125447 Test Loss 11.321642783467253
2325 Train Loss 8.124542 Test Loss 11.322627976240788
2326 Train Loss 8.123844 Test Loss 11.323497665474886
2327 Train Loss 8.123303 Test Loss 11.323913741205326
2328 Train Loss 8.122844 Test Loss 11.323784064931461
2329 Train Loss 8.122371 Test Loss 11.323614972130008
2330 Train Loss 8.121821 Test Loss 11.323354080159477
2331 Train Loss 8.12113 Test Loss 11.323503029778273
2332 Train Loss 8.122086 Test Loss 11.324075646862243
2333 Train Loss 8.120963 Test Loss 11.323663627115518
2334 Train Loss 8.120545 Test Lo

2472 Train Loss 8.020341 Test Loss 11.219511225435483
2473 Train Loss 8.019003 Test Loss 11.216858188148692
2474 Train Loss 8.017872 Test Loss 11.213729362212426
2475 Train Loss 8.016501 Test Loss 11.20869941797677
2476 Train Loss 8.014666 Test Loss 11.200355467256374
2477 Train Loss 8.013277 Test Loss 11.195499471477037
2478 Train Loss 8.012108 Test Loss 11.197359036230695
2479 Train Loss 8.011192 Test Loss 11.197794019343176
2480 Train Loss 8.010256 Test Loss 11.19702466294921
2481 Train Loss 8.009211 Test Loss 11.194759373874406
2482 Train Loss 8.008387 Test Loss 11.192756202099627
2483 Train Loss 8.007017 Test Loss 11.191275133739252
2484 Train Loss 8.005222 Test Loss 11.189620275392736
2485 Train Loss 8.003889 Test Loss 11.188874851113631
2486 Train Loss 8.001168 Test Loss 11.186456756154078
2487 Train Loss 8.002031 Test Loss 11.183525160103084
2488 Train Loss 7.9999123 Test Loss 11.185156356449058
2489 Train Loss 8.027854 Test Loss 11.17794577724014
2490 Train Loss 7.9995584 Test

2625 Train Loss 7.7343535 Test Loss 10.867032242644743
2626 Train Loss 7.7338743 Test Loss 10.865935572953349
2627 Train Loss 7.733491 Test Loss 10.865861105325761
2628 Train Loss 7.73306 Test Loss 10.865185589643774
2629 Train Loss 7.7322316 Test Loss 10.863781919410666
2630 Train Loss 7.731023 Test Loss 10.862114323544
2631 Train Loss 7.729351 Test Loss 10.859593060160066
2632 Train Loss 7.7275486 Test Loss 10.857966165445053
2633 Train Loss 7.725975 Test Loss 10.856830552502306
2634 Train Loss 7.7238674 Test Loss 10.857599270060085
2635 Train Loss 7.7620115 Test Loss 10.853816000871886
2636 Train Loss 7.723014 Test Loss 10.857020537000034
2637 Train Loss 7.7206745 Test Loss 10.857861408955143
2638 Train Loss 7.7179995 Test Loss 10.85983851638503
2639 Train Loss 7.7164607 Test Loss 10.859282738094592
2640 Train Loss 7.7150874 Test Loss 10.855519709175775
2641 Train Loss 7.7137036 Test Loss 10.855378780888412
2642 Train Loss 7.712631 Test Loss 10.853353685667328
2643 Train Loss 7.7116

KeyboardInterrupt: 

In [ ]:
PINN.W2

In [ ]:
u_pred = PINN.test(xt_test_tensor)

cmap = plt.cm.rainbow
plt.imshow(u_pred.reshape(100,256),cmap = cmap,aspect =1,vmin=-10,vmax=10)

In [ ]:
X_u_train

In [ ]:
np.min(u_pred)

In [ ]:
np.max(u_pred)

In [ ]:
cmap = plt.cm.rainbow
plt.imshow(u_true.reshape(100,256),cmap = cmap,aspect = 1,vmin=-10,vmax=10)

In [ ]:
np.min(u_true)

In [ ]:
cmap = plt.cm.rainbow
plt.imshow(np.transpose(np.abs(u_pred - u_true).reshape(100,256)),cmap = cmap,aspect = 0.2)

In [ ]:
a = 0 
for i in range(10):
    print(test_loss_full[i][-1])
    a = a + test_loss_full[i][-1]
print(a/10)